In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os

#os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

# Set GPU memory growth
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Dataset
---

In [3]:
# ImageDataGenerator
# ------------------
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#directories used to handle the datasets
dataset_dir = os.path.join(cwd, 'Segmentation_Dataset')
training_dir = os.path.join(dataset_dir, 'training')
validation_dir = os.path.join(dataset_dir, 'validation')

In [ ]:
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)

In [4]:
images_dir = os.path.join(training_dir, 'images', 'img')
masks_dir = os.path.join(training_dir, 'masks', 'img')

In [ ]:
#Validation split in a reproducible way
#Done according to https://cs230-stanford.github.io/train-dev-test-split.html#have-a-reproducible-script

for subdir, dirs, files in os.walk(images_dir):
    filenames = os.listdir(images_dir)
    np.random.shuffle(filenames)
    split = int(0.8 * len(filenames))
    valid_filenames = filenames[split:]
    if not os.path.exists(os.path.join(validation_dir, 'images')):
        os.mkdir(os.path.join(validation_dir, 'images'))
        os.mkdir(os.path.join(validation_dir, 'masks'))
        os.mkdir(os.path.join(validation_dir, 'images', 'img'))
        os.mkdir(os.path.join(validation_dir, 'masks', 'img'))
    for file in valid_filenames:
        shutil.move(os.path.join(images_dir, file), os.path.join(validation_dir, 'images', 'img', file))
        shutil.move(os.path.join(masks_dir, file), os.path.join(validation_dir, 'masks', 'img', file))

In [5]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = False

# Create training ImageDataGenerator object
# We need two different generators for images and corresponding masks
if apply_data_augmentation:
    train_img_data_gen = ImageDataGenerator(rotation_range=10,
                                            width_shift_range=10,
                                            height_shift_range=10,
                                            zoom_range=0.3,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            fill_mode='constant',
                                            cval=0,
                                            rescale=1./255)
    train_mask_data_gen = ImageDataGenerator(rotation_range=10,
                                             width_shift_range=10,
                                             height_shift_range=10,
                                             zoom_range=0.3,
                                             horizontal_flip=True,
                                             vertical_flip=True,
                                             fill_mode='constant',
                                             cval=0)
else:
    train_img_data_gen = ImageDataGenerator(rescale=1./255)
    train_mask_data_gen = ImageDataGenerator()

# Create validation and test ImageDataGenerator objects
valid_img_data_gen = ImageDataGenerator(rescale=1./255)
valid_mask_data_gen = ImageDataGenerator()

In [6]:
# Create generators to read images from dataset directory
# -------------------------------------------------------

# Batch size
bs = 16

# img shape
img_h = 256
img_w = 256


# Training
# Two different generators for images and masks
# ATTENTION: here the seed is important!! We have to give the same SEED to both the generator
# to apply the same transformations/shuffling to images and corresponding masks
train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       seed=SEED)  
train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         seed=SEED)
train_gen = zip(train_img_gen, train_mask_gen)

# Validation
valid_img_gen = valid_img_data_gen.flow_from_directory(os.path.join(validation_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=False,
                                                       interpolation='bilinear',
                                                       seed=SEED)
valid_mask_gen = valid_mask_data_gen.flow_from_directory(os.path.join(validation_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs, 
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=False,
                                                         interpolation='bilinear',
                                                         seed=SEED)
valid_gen = zip(valid_img_gen, valid_mask_gen)

Found 6117 images belonging to 1 classes.
Found 6117 images belonging to 1 classes.
Found 1530 images belonging to 1 classes.
Found 1530 images belonging to 1 classes.


In [7]:
# Create Dataset objects
# ----------------------

# Training
# --------
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 3]))

def prepare_target(x_, y_):
    y_ = tf.cast(tf.expand_dims(y_[..., 0], -1), tf.int32)
    return x_, tf.where(y_ > 0, y_ - 254, y_)

train_dataset = train_dataset.map(prepare_target)

# Repeat
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 3]))
valid_dataset = valid_dataset.map(prepare_target)

# Repeat
valid_dataset = valid_dataset.repeat()

In [ ]:
# Let's test data generator
# -------------------------
import time
import matplotlib.pyplot as plt

%matplotlib notebook

fig, ax = plt.subplots(1, 2)
fig.show()

# Assign a color to each class
colors_dict = {}
colors_dict[1] = [255, 255, 255]  # foreground
colors_dict[0] = [0, 0, 0]  # background

iterator = iter(train_dataset)

for _ in range(10):
    augmented_img, target = next(iterator)
    augmented_img = augmented_img[0]   # First element
    augmented_img = augmented_img * 255  # denormalize
    
    target = np.array(target[0, ..., 0])   # First element (squeezing channel dimension)
    
    # Assign colors (just for visualization)
    target_img = np.zeros([target.shape[0], target.shape[1], 3])
    
    target_img[np.where(target == 0)] = colors_dict[0]
    target_img[np.where(target == 1)] = colors_dict[1]
    
    ax[0].imshow(np.uint8(augmented_img))
    ax[1].imshow(np.uint8(target_img))
    
    fig.canvas.draw()
    time.sleep(1)

In [ ]:
np.unique(target_img)

In [19]:
def create_model(depth, start_f, num_classes, dynamic_input_shape):
    
    input_shape = [img_h, img_w, 3]
    
    inputs = tf.keras.layers.InputLayer(input_shape=input_shape)
    x = inputs
    encoder_layers = []
    input_shape=[None]
    for i in range(depth):
        if i == 0:
            if dynamic_input_shape:
                input_shape = [None, None, 3]
            else:
                input_shape = [img_h, img_w, 3]
        else:
            input_shape=[None]
        encoder_layers.append(x)
        x = tf.keras.layers.Conv2D(filters=start_f,
                                    kernel_size=(3, 3),
                                    strides=(1, 1),
                                    padding='same',
                                    input_shape=input_shape)(x)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.Conv2D(filters=start_f,
                                    kernel_size=(3, 3),
                                    strides=(1, 1),
                                    padding='same',
                                    input_shape=input_shape)(x)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
        start_f *= 2
        
    for conv in reversed(encoder_layers):
        start_f = start_f // 2
        x = tf.keras.layers.UpSampling2D(2, interpolation='bilinear')(x)
        x = tf.keras.layers.concatenate([x,conv])
        x = tf.keras.layers.Conv2D(filters=start_f,
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same')(x)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.Conv2D(filters=start_f,
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same')(x)
        x = tf.keras.layers.ReLU()(x)
        
    outputs = tf.keras.layers.Conv2D(filters=num_classes,
                                     kernel_size=(1, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     activation='sigmoid')
    
    model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])
    
    return model


In [ ]:
# Create Model
# ------------

def create_model(depth, start_f, num_classes, dynamic_input_shape):
    
    model = tf.keras.Sequential()
    
    # Encoder
    # -------
    for i in range(depth):
        
        
        model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same',
                                         input_shape=input_shape))
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same',
                                         input_shape=input_shape))
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

        start_f *= 2

    # Decoder
    # -------
    for i in range(depth):
        model.add(tf.keras.layers.UpSampling2D(2, interpolation='bilinear'))
        model.add(tf.keras.layers.Conv2D(filters=start_f // 2,
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same'))

        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.Conv2D(filters=start_f // 2,
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same'))

        model.add(tf.keras.layers.ReLU())

        start_f = start_f // 2

    # Prediction Layer
    # ----------------
    model.add(tf.keras.layers.Conv2D(filters=num_classes,
                                     kernel_size=(1, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     activation='sigmoid'))
    
    return model

In [20]:
model = create_model(depth=4, 
                     start_f=4, 
                     num_classes=2, 
                     dynamic_input_shape=False)

# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

AttributeError: 'InputLayer' object has no attribute 'shape'

In [ ]:
# Optimization params
# -------------------

# Loss
# Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False) 
# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------
def my_IoU(y_true, y_pred):
    # from pobability to predicted class {0, 1}
    y_pred = tf.cast(y_pred > 0.5, tf.float32) # when using sigmoid. Use argmax for softmax

    # A and B
    intersection = tf.reduce_sum(y_true * y_pred)
    # A or B
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    # IoU
    return intersection / union

# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=[my_IoU])

Training with callbacks
---

In [ ]:
import os
from datetime import datetime

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'segmentation_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=0)  # if 1 shows weights histograms
callbacks.append(tb_callback)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


history = model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_img_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_img_gen), 
          callbacks=callbacks)

fig, axs = plt.subplots(2)
axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title('Model loss')
axs[0].ylabel('Loss')
axs[0].xlabel('Epoch')
axs[0].legend(['Train', 'Valid'], loc='upper left')
axs[0].show()

axs[1].plot(history.history['my_IoU'])
axs[1].plot(history.history['my_IoU'])
axs[1].title('Intersection over Union')
axs[1].ylabel('IoU')
axs[1].xlabel('Epoch')
axs[1].legend(['Train', 'Valid'], loc='upper left')
axs[1].show()

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

Dataset Split Json creation
---

In [ ]:
import json

directories = os.listdir(training_dir)

dataset_split = {}

dataset_split["training"] =  os.listdir(images_dir)
                                                    
dataset_split["validation"] =  os.listdir(os.path.join(validation_dir, 'images', 'img'))
    
with open('dataset_split.json', 'w') as fp:
     json.dump(dataset_split, fp)

In [ ]:
for subdir, dirs, files in os.walk(os.path.join(validation_dir, 'images', 'img')):
    for file in files:
        shutil.move(os.path.join(subdir, file), os.path.join(images_dir, file))
        shutil.move(os.path.join(validation_dir, 'masks', 'img', file), os.path.join(masks_dir, file))
        
shutil.rmtree(validation_dir)

Compute Prediction
---

In [ ]:
import time
import matplotlib.pyplot as plt

from PIL import Image

%matplotlib notebook

# Cycle over test images
test_dir = os.path.join(dataset_dir, 'test')
test_img_dir = os.path.join(test_dir, 'images', 'img')

CSV File Creation
---

In [ ]:
import os
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(csv_fname, 'w') as f:

      f.write('ImageId,EncodedPixels,Width,Height\n')

      for key, value in results.items():
        value
        f.write(key + ',' + str(value) + ',' + '256' + ',' + '256' + '\n')

Image Flatten Codification
----

In [ ]:
def rle_encode(img):
    # Flatten column-wise
    pixels = img.T.flatten()
    pixels
    pixels = np.concatenate([[0], pixels, [0]])
    pixels
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs
    runs[1::2] -= runs[::2]
    runs
    return ' '.join(str(x) for x in runs)

In [ ]:
img_filenames = next(os.walk(test_img_dir))[2]

#fig, ax = plt.subplots(1, 2, figsize=(8, 8))
#fig.show()

# Assign a color to each class
colors_dict = {}
colors_dict[1] = [1] # foreground
colors_dict[0] = [0]      # background

results={}
i=1
for img_filename in img_filenames:
    
    
    img = Image.open(os.path.join(test_img_dir, img_filename))
    img = img.resize((256, 256))
    
    img_arr = np.expand_dims(np.array(img), 0)
    
    out_sigmoid = model.predict(x=img_arr / 255.)
    
    # Get predicted class as the index corresponding to the maximum value in the vector probability
    predicted_class = tf.argmax(out_sigmoid, -1)
    predicted_class = predicted_class[0]
    
    # Assign colors (just for visualization)
    prediction_img = np.zeros([predicted_class.shape[0], predicted_class.shape[1], 1])
    
    prediction_img[np.where(predicted_class == 0)] = colors_dict[0]
    prediction_img[np.where(predicted_class == 1)] = colors_dict[1]
    
    runs = rle_encode(prediction_img)
    results[img_filename[:-4]] = runs
    print(i)
    i+=1
    
    #ax[0].imshow(np.uint8(img_arr[0, ...]))
    #ax[1].imshow(np.uint8(prediction_img))
    
    #fig.canvas.draw()
    #time.sleep(2)

create_csv(results)    

Model Predictions
---

In [ ]:
img_filenames = next(os.walk(test_img_dir))[2]

fig, ax = plt.subplots(1, 3, figsize=(8, 8))
fig.show()

# Assign a color to each class
colors_dict = {}
colors_dict[0] = [255, 255, 255]  # foreground
colors_dict[1] = [0, 0, 0]      # background



for img_filename in img_filenames:
    
    mask_filename = img_filename[:-4] + '.png'
    
    img = Image.open(os.path.join(test_img_dir, img_filename))
    img = img.resize((256, 256))
    img_arr = np.expand_dims(np.array(img), 0)
    
    out_sigmoid = model.predict(x=img_arr / 255.)
    
    # Get predicted class as the index corresponding to the maximum value in the vector probability
    predicted_class = tf.argmax(out_sigmoid, -1)
    predicted_class = predicted_class[0]
    
    prediction_img = np.zeros([predicted_class.shape[0], predicted_class.shape[1], 3])
    
    prediction_img[np.where(predicted_class == 0)] = colors_dict[0]
    prediction_img[np.where(predicted_class == 1)] = colors_dict[1]

    runs = rle_encode(prediction_img)
    
    results[img_filename[:-4]] = runs
    
    # Assign colors (just for visualization)
    
    ax[0].imshow(np.uint8(img_arr[0, ...]))
    ax[1].imshow(np.uint8(target_img))
    ax[2].imshow(np.uint8(prediction_img))
    
    fig.canvas.draw()
    time.sleep(10)
    
